In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, KFold
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import itertools
import time
from sklearn.metrics import roc_auc_score, r2_score
from collections import Counter
import regex as re
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import path
import shap
import scipy
import mrmr
from scipy.stats import pearsonr
from scipy import stats
from sklearn import tree
from matplotlib.figure import Figure
import copy
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='first')

In [2]:
csv_path = os.getcwd()
csv_path

'/Users/saurabh/Desktop/HeartProjects/Exhalted/Params/Code_EXH/unexplained_dyspnea'

### Continuous value features

#### Data processing

In [3]:
# reading clinical features dataset
cpet_file = '../ExerciseMRIEvaluatio_DATA_LABELS_2021-03-16_1304.xlsx'
cpet_data = pd.read_excel(cpet_file)
cpet_data.dropna(subset=['VO2 (MAX):'],inplace=True)
cpet_data.dropna(subset=['VO2 %Predicted '],inplace=True)
cpet_data.shape

(67, 662)

In [4]:
# reading volume features
vol_df = pd.read_csv('vol.csv')
vol_df.drop(columns='Unnamed: 0',inplace=True);
vol_df.head()

,patient_id,lvmc_std,lvmc_snr,lvmc_vol_es,lvmc_vol_ed,lvmc_vol_max,lvmc_vol_min,lvmc_vol_es_equal_max,lvmc_vol_max_minus_min,lv_std,lv_snr,lv_vol_es,lv_vol_ed,stroke_volume,ejection_fraction
0,100B,2250.174975,14.478556,122988.1875,139436.2500,148709.4375,118114.6875,0,0.205735,2168.140010,20.616114,164954.4375,196361.4375,31407.0000,0.159945
1,101B,3389.251079,10.381979,136187.2500,123462.0000,136187.2500,96996.1875,1,0.287773,9517.657049,2.837491,39867.9375,133344.3750,93476.4375,0.701015
2,103,3343.955006,14.820243,175513.6875,144242.0625,175513.6875,144242.0625,1,0.178172,10586.318292,4.654657,104983.3125,203333.2500,98349.9375,0.483688
3,105B,2464.666301,20.048423,156019.6875,153582.9375,176055.1875,148777.1250,0,0.154940,10604.243561,5.340548,123597.3750,226617.7500,103020.3750,0.454600
4,109,2522.159368,18.556770,111413.6250,126507.9375,135239.6250,111413.6250,0,0.176176,15817.902632,2.284888,42440.0625,153447.5625,111007.5000,0.723423


In [5]:
# dropping patients with low LVEF
low_lv_ef = ['100B', '111', '114', '119', '123', '188']
vol_df = vol_df[~vol_df['patient_id'].isin(low_lv_ef)]
vol_df.shape

(51, 15)

In [6]:
# extracting patient IDs from clinical dataset that have corresponding 3D cardiac cine MR images
patient_ids = vol_df.patient_id.values
num_rows = len(patient_ids)
record_ids = []

for n in range(num_rows):
    pid = vol_df.iloc[n]['patient_id']
    num_pid = int(pid[0:3])
    record_ids.append(num_pid)

In [7]:
# filtering out clinical features for relevant patients
cpet_data = cpet_data[cpet_data['Record ID'].isin(record_ids)]

In [8]:
cpet_data.head()

,Record ID,LV Ejection Fraction,VO2 %Predicted,VO2 (MAX):,MRN,Event Name,Date:,Reviewed and subject acceptable for study?,HIV Positive?,PAH Patient?,...,Complete?.26,Medication List,Physician Signature.1,Complete,Complete?.27,Informed Consent.1,Complete?.28,Physician Signature.2,Complete.1,Complete?.29
1,101,60.0,64.0,1.52,1044315.0,Screening (Arm 1: Arm 1),2017-07-04,Yes,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103,55.0,43.4,2.36,1067466.0,Screening (Arm 1: Arm 1),2017-07-08,Yes,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,105,52.0,63.1,2.53,996643.0,Screening (Arm 1: Arm 1),2017-02-19,Yes,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109,70.0,44.0,1.23,1301694.0,Screening (Arm 1: Arm 1),2018-08-06,Yes,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,113,83.0,60.0,1.32,513421.0,Screening (Arm 1: Arm 1),2017-08-14,Yes,Yes,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
cpet_data.describe()

,LV Ejection Fraction,VO2 %Predicted,VO2 (MAX):,MRN,Body surface area (BSA),Systolic blood pressure,Diastolic blood pressure,Heart rate,Heart rate (peak),% of age predicted target heart rate,...,Stress MRI Peak A,Stress MRI Peak E' (high venc),Stress MRI Peak E' (low venc),Peak Rating of Perceived Exertion (RPE),Physician Signature,Medication List,Physician Signature.1,Complete,Physician Signature.2,Complete.1
count,26.000000,51.000000,51.000000,5.000000e+01,50.000000,51.000000,51.000000,50.000000,50.00000,50.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,60.307692,74.015686,1.866078,1.334633e+07,1.980600,125.627451,77.627451,75.040000,144.90000,87.740000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,8.776192,22.239815,0.599313,2.590012e+07,0.250782,14.093915,11.076030,12.885619,21.82888,10.001653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,45.000000,30.000000,0.570000,1.580150e+05,1.340000,94.000000,52.000000,47.000000,86.00000,65.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,55.000000,56.950000,1.525000,1.307721e+06,1.850000,119.500000,70.000000,66.000000,133.25000,82.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,59.500000,73.300000,1.830000,1.836710e+06,1.970000,124.000000,80.000000,74.500000,145.00000,86.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,64.000000,90.750000,2.265000,3.444585e+06,2.170000,132.000000,84.000000,85.750000,163.00000,94.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,83.000000,124.000000,3.270000,7.228645e+07,2.480000,166.000000,106.000000,102.000000,187.00000,108.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# dropping features with null values
cpet_data_filtered = cpet_data.dropna(axis='columns')
cpet_data_filtered.head()

,Record ID,VO2 %Predicted,VO2 (MAX):,Event Name,Date:,Reviewed and subject acceptable for study?,HIV Positive?,PAH Patient?,Complete?,Visit Number:,...,Classification: (choice=Pain),Classification: (choice=Pulmonary/Upper Respiratory),Classification: (choice=Renal/Genitourinary),Classification: (choice=Secondary Malignancy),Classification: (choice=Sexual/Reproductive Function),Classification: (choice=Syndromes),Classification: (choice=Vascular),Classification: (choice=Other),Complete?.20,Complete?.21
1,101,64.0,1.52,Screening (Arm 1: Arm 1),2017-07-04,Yes,Yes,No,Complete,visit 1,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Complete,Incomplete
2,103,43.4,2.36,Screening (Arm 1: Arm 1),2017-07-08,Yes,Yes,No,Complete,screening visit,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Complete,Incomplete
3,105,63.1,2.53,Screening (Arm 1: Arm 1),2017-02-19,Yes,Yes,No,Complete,screening visit,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Complete,Incomplete
4,109,44.0,1.23,Screening (Arm 1: Arm 1),2018-08-06,Yes,Yes,No,Complete,screening visit,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Complete,Incomplete
7,113,60.0,1.32,Screening (Arm 1: Arm 1),2017-08-14,Yes,Yes,No,Complete,screening visit,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Complete,Incomplete


In [11]:
# dropping columns with one single value
nunique = cpet_data_filtered.nunique()
cols_to_drop = nunique[nunique == 1].index
cpet_data_filtered.drop(cols_to_drop, axis=1, inplace = True)
cpet_data_filtered.head()

,Record ID,VO2 %Predicted,VO2 (MAX):,Date:,HIV Positive?,PAH Patient?,Visit Number:,Date and Time of ECG,Heart Rhythm (check all that apply): (choice=normal sinus rhythm),Heart Rhythm (check all that apply): (choice=sinus tachycardia),...,VO2/kg (MAX):,VCO2 (Max):,RER (MAX):,VEbtps (MAX):,VE/VO2 (MAX):,VE/VCO2 (MAX):,Date:.11,Any adverse events this cycle?,Classification: (choice=Musculoskeletal/Soft Tissue),Classification: (choice=Other)
1,101,64.0,1.52,2017-07-04,Yes,No,visit 1,2017-02-16 11:28:00,Checked,Unchecked,...,20.1,1.46,0.97,55.09,36.0,38.0,2017-02-18,No,Unchecked,Unchecked
2,103,43.4,2.36,2017-07-08,Yes,No,screening visit,2017-02-19 12:55:00,Unchecked,Unchecked,...,16.6,2.02,0.86,48.61,21.0,24.0,2017-02-27,No,Unchecked,Unchecked
3,105,63.1,2.53,2017-02-19,Yes,No,screening visit,2017-02-19 15:35:00,Checked,Unchecked,...,21.7,2.52,1.00,78.03,31.0,31.0,2017-03-05,No,Unchecked,Unchecked
4,109,44.0,1.23,2018-08-06,Yes,No,screening visit,2018-08-05 09:11:00,Unchecked,Checked,...,8.9,0.88,0.71,30.29,25.0,35.0,2018-08-07,No,Unchecked,Unchecked
7,113,60.0,1.32,2017-08-14,Yes,No,screening visit,2017-07-15 16:00:00,Checked,Unchecked,...,18.7,1.20,0.91,43.16,33.0,36.0,2017-08-20,No,Unchecked,Unchecked


In [12]:
# looking at 2 columns with same values
assert np.all(cpet_data_filtered['HIV Positive?.1'].values == cpet_data_filtered['HIV Positive?'].values)
assert np.all(cpet_data_filtered['PAH Patient?'].values == cpet_data_filtered['Pulmonary Hypertension?'].values)

In [13]:
# dropping unwanted columns
unwanted_columns = ['Date:', 'Visit Number:', 'Date and Time of ECG', 'Date of echo', 'Visit Number', \
                   'Date', 'Visit', 'Date:.1', 'Visit number:', 'Date of birth:', \
                   'HIV Positive?.1', 'Pulmonary Hypertension?', 'Date:.2', 'Visit #:', \
                   'Visit Date', 'Visit Number.1', 'Date:.3', 'Visit number:.1', \
                   'Visit number:.2', 'Date:.6', 'Visit number:.4', 'Date:.9', \
                   'Visit number:.7', 'Date of test:.1', 'Visit number:.9', \
                   'Weight:', 'Height:','Date:.11']
cpet_data_filtered.drop(unwanted_columns, axis=1, inplace = True)
cpet_data_filtered.head()

,Record ID,VO2 %Predicted,VO2 (MAX):,HIV Positive?,PAH Patient?,Heart Rhythm (check all that apply): (choice=normal sinus rhythm),Heart Rhythm (check all that apply): (choice=sinus tachycardia),Heart Rhythm (check all that apply): (choice=sinus bradycardia),Heart Rhythm (check all that apply): (choice=other),Systolic blood pressure,...,RER 1.0 Achieved?,VO2/kg (MAX):,VCO2 (Max):,RER (MAX):,VEbtps (MAX):,VE/VO2 (MAX):,VE/VCO2 (MAX):,Any adverse events this cycle?,Classification: (choice=Musculoskeletal/Soft Tissue),Classification: (choice=Other)
1,101,64.0,1.52,Yes,No,Checked,Unchecked,Unchecked,Unchecked,132.0,...,No,20.1,1.46,0.97,55.09,36.0,38.0,No,Unchecked,Unchecked
2,103,43.4,2.36,Yes,No,Unchecked,Unchecked,Checked,Unchecked,127.0,...,No,16.6,2.02,0.86,48.61,21.0,24.0,No,Unchecked,Unchecked
3,105,63.1,2.53,Yes,No,Checked,Unchecked,Unchecked,Unchecked,120.0,...,Yes,21.7,2.52,1.00,78.03,31.0,31.0,No,Unchecked,Unchecked
4,109,44.0,1.23,Yes,No,Unchecked,Checked,Unchecked,Unchecked,122.0,...,No,8.9,0.88,0.71,30.29,25.0,35.0,No,Unchecked,Unchecked
7,113,60.0,1.32,Yes,No,Checked,Unchecked,Unchecked,Unchecked,120.0,...,No,18.7,1.20,0.91,43.16,33.0,36.0,No,Unchecked,Unchecked


In [14]:
# extracting ground truth
vo2_pp = cpet_data_filtered['VO2 %Predicted ']
pkvo2 = cpet_data_filtered['VO2 (MAX):']
pkvo2_vals = pkvo2.values
response_vars = ['VO2 %Predicted ', 'VO2 (MAX):']
cpet_data_filtered.drop(response_vars, axis=1, inplace = True)

In [15]:
# dropping column 'HIV Positive?' since all patients have HIV
hiv_pos = cpet_data_filtered['HIV Positive?']
cpet_data_filtered.drop(['HIV Positive?'], axis=1, inplace = True)

In [16]:
cpet_data_filtered.head()

,Record ID,PAH Patient?,Heart Rhythm (check all that apply): (choice=normal sinus rhythm),Heart Rhythm (check all that apply): (choice=sinus tachycardia),Heart Rhythm (check all that apply): (choice=sinus bradycardia),Heart Rhythm (check all that apply): (choice=other),Systolic blood pressure,Diastolic blood pressure,Complete?.3,Forced Vital Capacity (Best),...,RER 1.0 Achieved?,VO2/kg (MAX):,VCO2 (Max):,RER (MAX):,VEbtps (MAX):,VE/VO2 (MAX):,VE/VCO2 (MAX):,Any adverse events this cycle?,Classification: (choice=Musculoskeletal/Soft Tissue),Classification: (choice=Other)
1,101,No,Checked,Unchecked,Unchecked,Unchecked,132.0,92.0,Complete,3.01,...,No,20.1,1.46,0.97,55.09,36.0,38.0,No,Unchecked,Unchecked
2,103,No,Unchecked,Unchecked,Checked,Unchecked,127.0,73.0,Complete,4.25,...,No,16.6,2.02,0.86,48.61,21.0,24.0,No,Unchecked,Unchecked
3,105,No,Checked,Unchecked,Unchecked,Unchecked,120.0,72.0,Complete,5.01,...,Yes,21.7,2.52,1.00,78.03,31.0,31.0,No,Unchecked,Unchecked
4,109,No,Unchecked,Checked,Unchecked,Unchecked,122.0,64.0,Complete,1.63,...,No,8.9,0.88,0.71,30.29,25.0,35.0,No,Unchecked,Unchecked
7,113,No,Checked,Unchecked,Unchecked,Unchecked,120.0,80.0,Complete,3.14,...,No,18.7,1.20,0.91,43.16,33.0,36.0,No,Unchecked,Unchecked


In [17]:
# let's first look at continuous value columns
cpet_data_float = cpet_data_filtered.select_dtypes(include=['float64'])
cpet_data_float.shape

(51, 48)

In [18]:
# dropping gas exchange or CPET features since it requires patients to exercise.
gas_exchange_features = []
gas_exchange_keywords = ['VCO2', 'VEbtps', 'VO2', 'FEV1', 'VCmax', 'FEF', 'RER']
for feat in cpet_data_float.columns.values:
    for keyword in  gas_exchange_keywords:
        if keyword in feat: gas_exchange_features.append(feat)
cpet_data_float_no_gas = cpet_data_float.drop(gas_exchange_features, axis=1)

In [19]:
# looking at remaining columns
cpet_data_float_no_gas.columns.values

array(['Systolic blood pressure', 'Diastolic blood pressure',
       'Forced Vital Capacity (Best)', 'Forced Vital Capacity (% Pred)',
       'Age', 'Number of Inhlaers', 'Number of hypertension medications',
       'White Blood Cell Count', 'Red Blood Cell Count', 'Hemoglobin',
       'Hematocrit', 'MCV', 'MCHC', 'Platelet Count', 'RDW',
       'Neutrophils', 'Lymphocytes', 'Monocytes', 'Eosinophils',
       'Basophils', 'Absolute Neutrophils', 'Absolute Lymphocytes',
       'Absolute Monocytes', 'Absolute Eosinophils', 'Absolute Basophils',
       'Age:', 'Height (cm):', 'Weight (kg):', 'BMI'], dtype=object)

In [20]:
# dropping remaining gas features
cpet_data_float_no_gas = cpet_data_float_no_gas.drop(['Forced Vital Capacity (Best)', 
                                                      'Forced Vital Capacity (% Pred)'],\
                                                     axis=1)
cpet_data_float_no_gas.shape

(51, 27)

In [21]:
# looking at correlation of features with PkVO2
response = pkvo2.values
predictors = cpet_data_float_no_gas.columns.values
pearson_r_vals = []
for predictor in predictors:
    pearson_val = pearsonr(cpet_data_float_no_gas[predictor],response)
    pearson_r_vals.append((predictor, round(pearson_val[0]**2,3), round(pearson_val[1],6)))
[print(ent) for ent in sorted(pearson_r_vals, key=lambda x: x[1], reverse=True)];

('Height (cm):', 0.447, 0.0)
('Weight (kg):', 0.174, 0.002328)
('Hemoglobin', 0.163, 0.003269)
('Red Blood Cell Count', 0.151, 0.004814)
('Hematocrit', 0.131, 0.009046)
('Neutrophils', 0.043, 0.144744)
('MCV', 0.037, 0.177588)
('Lymphocytes', 0.033, 0.203778)
('Absolute Lymphocytes', 0.029, 0.234237)
('Age:', 0.028, 0.243881)
('RDW', 0.024, 0.277713)
('Absolute Neutrophils', 0.024, 0.27874)
('MCHC', 0.019, 0.339123)
('Monocytes', 0.016, 0.372065)
('Absolute Eosinophils', 0.013, 0.423196)
('Absolute Basophils', 0.012, 0.449744)
('Platelet Count', 0.01, 0.482653)
('Diastolic blood pressure', 0.009, 0.513975)
('Absolute Monocytes', 0.007, 0.572677)
('BMI', 0.007, 0.566548)
('Eosinophils', 0.006, 0.581379)
('Basophils', 0.006, 0.579294)
('Age', 0.005, 0.607298)
('Number of Inhlaers', 0.005, 0.637741)
('Systolic blood pressure', 0.004, 0.675959)
('White Blood Cell Count', 0.004, 0.66021)
('Number of hypertension medications', 0.0, 0.925073)


In [22]:
# adding patient ids to records
cpet_data_float_no_gas['patient_id'] = patient_ids

In [23]:
# saving important clinical features in csv
cpet_data_float_no_gas.to_csv('important_clinical_features.csv')

In [24]:
# feature scaling
numerical_columns = list(cpet_data_float_no_gas.columns.values)
numerical_columns.remove('patient_id')
cpet_data_float_no_gas_scaled = cpet_data_float_no_gas.copy(deep=True)
cpet_data_float_no_gas_scaled[numerical_columns] = scaler.fit_transform(cpet_data_float_no_gas_scaled[numerical_columns])
cpet_data_float_no_gas_scaled.reset_index(inplace=True, drop=True)
cpet_data_float_no_gas_scaled.head()

,Systolic blood pressure,Diastolic blood pressure,Age,Number of Inhlaers,Number of hypertension medications,White Blood Cell Count,Red Blood Cell Count,Hemoglobin,Hematocrit,MCV,...,Absolute Neutrophils,Absolute Lymphocytes,Absolute Monocytes,Absolute Eosinophils,Absolute Basophils,Age:,Height (cm):,Weight (kg):,BMI,patient_id
0,0.527778,0.740741,0.818182,0.000000,0.0,0.142857,0.629091,0.689655,0.619910,0.425,...,0.068323,0.099905,0.277108,0.196970,0.250,0.733333,0.404255,0.312245,0.232509,101B
1,0.458333,0.388889,0.575758,0.000000,0.0,0.047619,0.774545,0.879310,0.859729,0.500,...,0.030021,1.000000,0.192771,0.212121,0.125,0.377778,0.468085,1.000000,0.892617,103
2,0.361111,0.370370,0.712121,0.666667,0.0,0.152381,0.316364,0.482759,0.438914,0.625,...,0.108696,0.071361,0.192771,0.348485,0.375,0.577778,0.787234,0.733673,0.444200,105B
3,0.388889,0.222222,0.696970,0.666667,0.0,1.000000,0.236364,0.241379,0.321267,0.600,...,1.000000,0.104662,0.216867,0.015152,0.500,0.555556,0.276596,0.948980,1.000000,109
4,0.361111,0.518519,0.833333,0.000000,0.0,0.133333,0.625455,0.603448,0.552036,0.350,...,0.128364,0.029496,0.265060,0.166667,0.750,0.755556,0.085106,0.265306,0.335454,113B


### Model prediction

Let's predict PkVO2 from clinical features. We will limit ourselves to 6 features since any number above that will cause overfitting.

In [25]:
def adj_r2_score(r2, k):
    '''
    returns adjusted R2
    '''
    n = 51
    return 1. - ((1.-r2)*(n-1.)/(n-k-1))

In [26]:
def cross_val_5fold(df_scaled, response, hand_selected_features):
    '''
    performs 5-fold cross validation
    '''
    features_subset = hand_selected_features
    X = df_scaled[features_subset]
    y = response
    # Number of random trials
    NUM_TRIALS = 20
    n_splits = 5

    lin_reg = LinearRegression()

    # Arrays to store scores
    val_r2_scores = np.zeros(NUM_TRIALS*n_splits)
    train_r2_scores = np.zeros(NUM_TRIALS*n_splits)
    val_adj_r2_scores = np.zeros(NUM_TRIALS*n_splits)
    train_adj_r2_scores = np.zeros(NUM_TRIALS*n_splits)
    val_R_scores = np.zeros(NUM_TRIALS*n_splits)
    train_R_scores = np.zeros(NUM_TRIALS*n_splits)
    coeffs = []
    intcepts = []

    for i in range(NUM_TRIALS):
        val_preds_per_trial = []

        cv = KFold(n_splits=n_splits, shuffle=True, random_state=i)

        iteration = 0
        for train_ix, test_ix in cv.split(X,y):
            X_train, X_test = X.loc[train_ix, :], X.loc[test_ix, :]
            y_train, y_test = y[train_ix], y[test_ix]
            lin_reg.fit(X_train, y_train)
            y_hat = lin_reg.predict(X_test)
            val_preds_per_trial.append([df_scaled.loc[test_ix]['patient_id'].values, y_test, y_hat])
            
            val_r2_scores[i*n_splits + iteration] = r2_score(y_test, y_hat)    
            train_r2_scores[i*n_splits + iteration] = r2_score(y_train, lin_reg.predict(X_train))
            val_adj_r2_scores[i*n_splits + iteration] = adj_r2_score(r2_score(y_test, y_hat), \
                                                                     len(hand_selected_features))    
            train_adj_r2_scores[i*n_splits + iteration] = adj_r2_score(r2_score(y_train, lin_reg.predict(X_train)),\
                                                                       len(hand_selected_features))
            
            val_R_scores[i*n_splits + iteration] = pearsonr(y_test, y_hat)[0]    
            train_R_scores[i*n_splits + iteration] = pearsonr(y_train, lin_reg.predict(X_train))[0]  
            
            coeffs.append(lin_reg.coef_)
            intcepts.append(lin_reg.intercept_)
            
            iteration += 1
    sqrt_n = np.sqrt(NUM_TRIALS*n_splits)
    print('train scores: mean, std error', round(train_r2_scores.mean(), 3), round(train_r2_scores.std()/sqrt_n, 2))
    print('validation scores: mean, std error', round(val_r2_scores.mean(), 3), round(val_r2_scores.std()/sqrt_n, 2))
    
    return val_r2_scores.mean(), val_r2_scores.std()/sqrt_n,\
train_r2_scores.mean(), train_r2_scores.std()/sqrt_n,\
intcepts,coeffs,\
val_R_scores.mean(), val_R_scores.std()/sqrt_n,\
train_R_scores.mean(), train_R_scores.std()/sqrt_n


In [27]:
def interesting_combos_func(df_scaled, numerical_columns, num_feat, shortlisted_features, interesting_combos):
    '''
    returns subsets of features with low VIF/multicollinearity
    '''
    rem_feat = num_feat - len(shortlisted_features)
    rem_columns = list(copy.deepcopy(numerical_columns))
    for feat in shortlisted_features:
        rem_columns.remove(feat) 

    for comb in itertools.combinations(rem_columns, rem_feat):
        important_features_subset = []
        important_features_subset = list(comb)
        important_features_subset.extend(shortlisted_features)
        X = df_scaled[important_features_subset].assign(const=1)
        vif_values = [vif(X.values, i) for i in range(num_feat)]
        if max(vif_values) <= 5.:
            vif_feat_vals = {important_features_subset[ind]: vals for ind,vals in enumerate(vif_values)}
            interesting_combos.append(important_features_subset)
    return interesting_combos

def best_features_subset(df_scaled, response, interesting_combos):
    '''
    returns subset which best predicts PkVO2 and the best R^2 value
    '''
    max_reg = 0.
    multivariate_y = response
    for important_features_subset in interesting_combos:
        multivariate_x = df_scaled[important_features_subset]
        reg = LinearRegression().fit(multivariate_x, multivariate_y)
        reg_score = reg.score(multivariate_x, multivariate_y)
        if reg_score > max_reg:
            max_reg = reg_score
            max_reg_features = important_features_subset
    if interesting_combos == []: return [], 0.

    return max_reg_features, max_reg

In [31]:
def feature_iterations(initial_feature, df, numerical_columns, num_iters):
    '''
    performs cross validation for feature subsets of increasing size
    '''
    max_reg_features = initial_feature
    num_feat = len(max_reg_features)

    r2_scores_mean = []
    r2_scores_std = []
    num_feats = []

    for i in range(num_iters):
        num_feat += 1 
        interesting_combos = []
        shortlisted_features =  max_reg_features
        #
        interesting_combos = interesting_combos_func(df, numerical_columns, num_feat, shortlisted_features, interesting_combos)
        print('\nnumber of features:', num_feat, ', number of valid subsets:', len(interesting_combos))
        #
        max_reg_features, max_reg = best_features_subset(df, pkvo2_vals, interesting_combos)
        if not max_reg_features == []:
            #
            val_r2_scores_mean, val_r2_scores_std,\
            train_r2_scores_mean, train_r2_scores_std,\
            intcepts,coeffs,\
            val_R_scores_mean, val_R_scores_std,\
            train_R_scores_mean, train_R_scores_std = cross_val_5fold(df, pkvo2_vals, max_reg_features)
            r2_scores_mean.append(val_r2_scores_mean)
            r2_scores_std.append(val_r2_scores_std)  
        else:
            r2_scores_mean.append(0.)
            r2_scores_std.append(0.)        
        num_feats.append(num_feat)
    return r2_scores_mean, r2_scores_std, num_feats, max_reg_features, intcepts,coeffs

In [32]:
def average_model_predictions(df, features, y, coeffs, intercepts):
    '''
    returns predictions from average model of 100 iterations
    '''
    mean_coeffs = np.mean(coeffs,axis=0)
    mean_coeffs = np.array(mean_coeffs).reshape((mean_coeffs.shape[0],1))
    mean_intcept = np.mean(intercepts,axis=0)
    X = df[features]
    ans = np.matmul(X, mean_coeffs)
    
    ground_truth = y
    feat_x_X = np.matmul(X, mean_coeffs)
    mean_response = feat_x_X + mean_intcept

    return mean_response.values
    

In [33]:
# iteratively increasing the feature subset by allowing one more feature in each iteration.
# let's start with height since it has the highest correlation with PkVO2
initial_feature = ['Height (cm):']
r2_scores_mean, r2_scores_std, num_feats, max_reg_features, intcepts, coeffs = \
feature_iterations(initial_feature, cpet_data_float_no_gas_scaled, numerical_columns, 5)


number of features: 2 , number of valid subsets: 26
train scores: mean, std error 0.495 0.01
validation scores: mean, std error 0.312 0.04

number of features: 3 , number of valid subsets: 24
train scores: mean, std error 0.537 0.01
validation scores: mean, std error 0.343 0.04

number of features: 4 , number of valid subsets: 23
train scores: mean, std error 0.56 0.01
validation scores: mean, std error 0.335 0.04

number of features: 5 , number of valid subsets: 22
train scores: mean, std error 0.583 0.0
validation scores: mean, std error 0.212 0.06

number of features: 6 , number of valid subsets: 21
train scores: mean, std error 0.595 0.0
validation scores: mean, std error 0.274 0.05


In [34]:
# making predictions from the average model
ground_truth = pkvo2.values
predicted = average_model_predictions(cpet_data_float_no_gas_scaled, max_reg_features, \
                                      ground_truth, coeffs, intcepts)

In [35]:
print('R^2 score =', round(r2_score(ground_truth, predicted),2))

R^2 score = 0.59


In [36]:
print('selected features:')
max_reg_features

selected features:


['Absolute Basophils',
 'Absolute Lymphocytes',
 'RDW',
 'MCV',
 'Lymphocytes',
 'Height (cm):']

In [37]:
# verifying that there is no multicollinearity between features selected
significant_features = max_reg_features
X = cpet_data_float_no_gas_scaled[significant_features].assign(const=1)
vif_values = [vif(X.values, i) for i in range(len(significant_features))]
vif_feat_vals = {significant_features[ind]: vals for ind,vals in enumerate(vif_values)}
vif_feat_vals = sorted(vif_feat_vals.items(), key = lambda item: item[1], reverse = True)
print('VIF values: (feature, value)\n')
[print(ent[0], round(ent[1],3)) for ent in vif_feat_vals];

VIF values: (feature, value)

RDW 1.479
MCV 1.414
Lymphocytes 1.224
Absolute Lymphocytes 1.218
Absolute Basophils 1.055
Height (cm): 1.048


### Categorical Columns

#### Preprocessing

In [38]:
# extracting categorical columns
cpet_data_string = cpet_data_filtered.select_dtypes(include=['object'])
cpet_data_string.head()

,Record ID,PAH Patient?,Heart Rhythm (check all that apply): (choice=normal sinus rhythm),Heart Rhythm (check all that apply): (choice=sinus tachycardia),Heart Rhythm (check all that apply): (choice=sinus bradycardia),Heart Rhythm (check all that apply): (choice=other),Complete?.3,What is your current gender identity?,Race (self-reported): More than one race may be marked (choice=White or Caucasian),Race (self-reported): More than one race may be marked (choice=Black or African American),...,Abdomen:,Neurological:,Musculoskeletal:,Other:,Complete?.16,Sex:,RER 1.0 Achieved?,Any adverse events this cycle?,Classification: (choice=Musculoskeletal/Soft Tissue),Classification: (choice=Other)
1,101,No,Checked,Unchecked,Unchecked,Unchecked,Complete,Male,Unchecked,Checked,...,Normal,Normal,Normal,Normal,Complete,Male,No,No,Unchecked,Unchecked
2,103,No,Unchecked,Unchecked,Checked,Unchecked,Complete,Male,Unchecked,Checked,...,Normal,Normal,Normal,Not done,Complete,Male,No,No,Unchecked,Unchecked
3,105,No,Checked,Unchecked,Unchecked,Unchecked,Complete,Male,Unchecked,Checked,...,Normal,Normal,Normal,Not done,Complete,Male,Yes,No,Unchecked,Unchecked
4,109,No,Unchecked,Checked,Unchecked,Unchecked,Complete,Female,Unchecked,Checked,...,Normal,Normal,Normal,Normal,Complete,Female,No,No,Unchecked,Unchecked
7,113,No,Checked,Unchecked,Unchecked,Unchecked,Complete,Male,Unchecked,Checked,...,Normal,Normal,Normal,Not done,Complete,Male,No,No,Unchecked,Unchecked


In [39]:
# dropping unwanted columns
cpet_data_string.drop(['Record ID','Complete?.3','Confirmed HIV','RER 1.0 Achieved?','Complete?.16',],axis=1,inplace=True)
cpet_data_string.head()

,PAH Patient?,Heart Rhythm (check all that apply): (choice=normal sinus rhythm),Heart Rhythm (check all that apply): (choice=sinus tachycardia),Heart Rhythm (check all that apply): (choice=sinus bradycardia),Heart Rhythm (check all that apply): (choice=other),What is your current gender identity?,Race (self-reported): More than one race may be marked (choice=White or Caucasian),Race (self-reported): More than one race may be marked (choice=Black or African American),Race (self-reported): More than one race may be marked (choice=Other (specify)),Employment Status,...,Cardiovascular:,Respiratory:,Abdomen:,Neurological:,Musculoskeletal:,Other:,Sex:,Any adverse events this cycle?,Classification: (choice=Musculoskeletal/Soft Tissue),Classification: (choice=Other)
1,No,Checked,Unchecked,Unchecked,Unchecked,Male,Unchecked,Checked,Unchecked,No,...,Normal,Normal,Normal,Normal,Normal,Normal,Male,No,Unchecked,Unchecked
2,No,Unchecked,Unchecked,Checked,Unchecked,Male,Unchecked,Checked,Unchecked,No,...,Normal,Normal,Normal,Normal,Normal,Not done,Male,No,Unchecked,Unchecked
3,No,Checked,Unchecked,Unchecked,Unchecked,Male,Unchecked,Checked,Unchecked,Yes,...,Normal,Normal,Normal,Normal,Normal,Not done,Male,No,Unchecked,Unchecked
4,No,Unchecked,Checked,Unchecked,Unchecked,Female,Unchecked,Checked,Unchecked,No,...,Normal,Abnormal,Normal,Normal,Normal,Normal,Female,No,Unchecked,Unchecked
7,No,Checked,Unchecked,Unchecked,Unchecked,Male,Unchecked,Checked,Unchecked,No,...,Normal,Normal,Normal,Normal,Normal,Not done,Male,No,Unchecked,Unchecked


Let's remove columns that contain pulmonary hypertension (PAH) information since they are predictive of low PkVO2 values. Additionially PAH is diagnosed by echo which can be expensive.

Additionally, let's also drop exercise related features.

In [40]:
interesting_categorical_cols = [
        #'PAH Patient?', \
        'Race (self-reported):  More than one race may be marked (choice=White or Caucasian)',\
       'Race (self-reported):  More than one race may be marked (choice=Black or African American)',\
       'Race (self-reported):  More than one race may be marked (choice=Other (specify))',\
       'Current smoking:', 'Past alcohol use:',\
       'Past alcohol use:', 'Past alcohol use:',\
       'Current illicit drug use:',\
       'Family history for cardiovascular disease (CVD) (family medical history of CVD in first degree relative):',\
       'Diagnosis of cancer:', 'History of pregnancy:',\
       #'History of hypertension:', 
        'Diabetes:',\
       'Hyperlipidemia:', 'Coronary Artery Disease (CAD):',\
       "If 'Yes' to CAD, have you had? (choice=Prior myocardial infarction (MI))",\
       "If 'Yes' to CAD, have you had? (choice=Prior percutaneous coronary intervention (PCI))",\
       "If 'Yes' to CAD, have you had? (choice=Angina with exertion)",\
       'History of stroke?',\
       'Chronic kidney disease (CKD)?',\
       'What is your ambulatory status?',\
       'Inhaler', 'Nasal Suspension', 'Infusion',\
       'Oral Vasodilator ', 'Statins', 'Asprin',\
       'B-Blockers', 'Diuretics', 'Calcium Channel Blockers',\
       'ACE Inhibitors', 'ARB', 'HEENT:', 'Neck and Thyroid:', 'Skin:',
       'Cardiovascular:', 'Respiratory:', 'Abdomen:',
       'Neurological:', 'Musculoskeletal:', 'Other:',
       'Other:', 'Sex:',                       
        ]

In [41]:
# keeping important columns only
cpet_data_string = cpet_data_string[interesting_categorical_cols]

In [42]:
# one hot encoding of important clinical features
cpet_data_categorical = enc.fit_transform(cpet_data_string)
categorical_col_names = enc.get_feature_names_out(input_features = cpet_data_string.columns.values)
cpet_data_categorical_df = pd.DataFrame(data = cpet_data_categorical.toarray(), columns = list(categorical_col_names))
cpet_data_categorical_df.head()

,Race (self-reported): More than one race may be marked (choice=White or Caucasian)_Unchecked,Race (self-reported): More than one race may be marked (choice=Black or African American)_Unchecked,Race (self-reported): More than one race may be marked (choice=Other (specify))_Unchecked,Current smoking:_Yes,Past alcohol use:_Occasional,Past alcohol use:_Regular,Past alcohol use:_Seldom,Past alcohol use:_Occasional,Past alcohol use:_Regular,Past alcohol use:_Seldom,...,Cardiovascular:_Normal,Respiratory:_Normal,Abdomen:_Normal,Neurological:_Normal,Musculoskeletal:_Normal,Other:_Normal,Other:_Not done,Other:_Normal,Other:_Not done,Sex:_Male
0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
1,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0


In [43]:
# combining categorical and continuous features
combined_shorlist_df = pd.concat([cpet_data_float_no_gas_scaled, cpet_data_categorical_df], axis=1)
combined_shorlist_df.head()

,Systolic blood pressure,Diastolic blood pressure,Age,Number of Inhlaers,Number of hypertension medications,White Blood Cell Count,Red Blood Cell Count,Hemoglobin,Hematocrit,MCV,...,Cardiovascular:_Normal,Respiratory:_Normal,Abdomen:_Normal,Neurological:_Normal,Musculoskeletal:_Normal,Other:_Normal,Other:_Not done,Other:_Normal,Other:_Not done,Sex:_Male
0,0.527778,0.740741,0.818182,0.000000,0.0,0.142857,0.629091,0.689655,0.619910,0.425,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
1,0.458333,0.388889,0.575758,0.000000,0.0,0.047619,0.774545,0.879310,0.859729,0.500,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2,0.361111,0.370370,0.712121,0.666667,0.0,0.152381,0.316364,0.482759,0.438914,0.625,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
3,0.388889,0.222222,0.696970,0.666667,0.0,1.000000,0.236364,0.241379,0.321267,0.600,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
4,0.361111,0.518519,0.833333,0.000000,0.0,0.133333,0.625455,0.603448,0.552036,0.350,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0


In [44]:
# adding patient id to combined dataframe
combined_shorlist_df['patient_id'] = patient_ids    

In [45]:
# saving dataframe to csv file
combined_shorlist_df.to_csv('interesting_clinical_features.csv')

In [46]:
# extracting numerical columns
numerical_columns = list(combined_shorlist_df.columns.values)
numerical_columns.remove('patient_id')

In [47]:
# searching for the best combination of features starting with height.
initial_features = [
'Height (cm):'
            ]
num_iters = 4
r2_scores_mean, r2_scores_std, num_feats, max_reg_features, intcepts, coeffs = \
feature_iterations(initial_features, combined_shorlist_df, numerical_columns, num_iters)


number of features: 2 , number of valid subsets: 64
train scores: mean, std error 0.52 0.0
validation scores: mean, std error 0.309 0.04

number of features: 3 , number of valid subsets: 63
train scores: mean, std error 0.562 0.01
validation scores: mean, std error 0.323 0.04

number of features: 4 , number of valid subsets: 61
train scores: mean, std error 0.594 0.0
validation scores: mean, std error 0.361 0.04

number of features: 5 , number of valid subsets: 60
train scores: mean, std error 0.655 0.0
validation scores: mean, std error 0.395 0.04


In [48]:
# looking at the best set of clinical features
print('best features:')
[print(ent) for ent in max_reg_features];

best features:
Age:
Family history for cardiovascular disease (CVD) (family medical history of CVD in first degree relative):_Yes
Lymphocytes
Musculoskeletal:_Normal
Height (cm):


In [49]:
# predicting pkvo2 from best subset
predicted = average_model_predictions(combined_shorlist_df, max_reg_features, pkvo2.values, coeffs, intcepts)

In [50]:
print('R^2 score from top 6 clinical features:')
print(round(r2_score(ground_truth, predicted), 2))

R^2 score from top 6 clinical features:
0.65


In [51]:
pearson_r = pearsonr(ground_truth, predicted)
print('Pearson R score & p-value from top 6 clinical features:')
print(round(pearson_r[0][0],2), "{:.0e}".format(pearson_r[1]))

Pearson R score & p-value from top 6 clinical features:
0.8 1e-12
